

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


## OpenTripPlanner (OTP) walkthrough

1. Install Java SDK.
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/).
3. Download gtfs.zip of a given city into a `folder` from `transit.land`.
4. Download osm.pbf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token).
5. Call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files.
6. If succesful you shall be able to open a web search in your browser: [`http://localhost:8080/`](http://localhost:8080/).

information from [link](http://docs.opentripplanner.org/en/latest/Basic-Tutorial/)


In [1]:
import pandas as pd
import requests

### Local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

The `georequests.csv` file contains requests to travel from specific origin to destination points. 

In [3]:
PATH = 'georequests.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

The `origin_x`, `destination_x`, `origin_y` and `destination_y` represent the longitude (x) and latitude (y) coordinates of the origin and destination locations

In [4]:
df = pd.read_csv(PATH, index_col=[0])  # load the csv
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
618,4.865235,52.364162,4.881339,52.385118,2020-10-29 15:26:25
328,4.861646,52.366034,4.917393,52.358030,2020-10-29 14:53:47
773,4.803047,52.390442,4.868713,52.369369,2020-10-29 14:38:21
151,4.785410,52.355865,4.874722,52.377780,2020-10-29 14:43:10
686,4.908205,52.341057,4.870084,52.352835,2020-10-29 15:05:50


We generate a new DataFrame containing the details of our request, including the designated origin and destination for the journey.

In [5]:
df1 = pd.DataFrame(columns=['origin_x','origin_y','destination_x','destination_y','treq'])
df1['origin_x'] = [16.89423, 16.91294, 16.94984]
df1['origin_y'] = [52.40504, 52.42797, 52.41404]

df1['destination_x'] = [16.94984, 16.92324, 16.88410]
df1['destination_y'] = [52.40619, 52.39833, 52.36973]

df1['treq'] = pd.Timestamp(2023, 3, 7, 12)

df =df1
df

,origin_x,origin_y,destination_x,destination_y,treq
0,16.89423,52.40504,16.94984,52.40619,2023-03-07 12:00:00
1,16.91294,52.42797,16.92324,52.39833,2023-03-07 12:00:00
2,16.94984,52.41404,16.88410,52.36973,2023-03-07 12:00:00


### Sample query 

before you can query the server you need to run it -see `run OTP server.ipynb` in this folder

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [6]:
row = df.sample(1).squeeze() #randomly choose a row from df and convert it into a Pandas Series
query = make_query(row)
query

{'fromPlace': '52.40504,16.89423',
 'toPlace': '52.40619,16.94984',
 'time': '00:0pm',
 'date': '3-7-2023',
 'mode': 'TRANSIT,WALK',
 'maxWalkDistance': 2000,
 'arriveBy': 'false'}

The itinerary refers to a possible travel plan. 

In [7]:
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 1939,
 'walkDistance': 1038.213052617993,
 'transfers': 0,
 'transitTime': 1080,
 'waitingTime': 2,
 'modes': [['WALK', 361, 437], ['TRAM', 1080, 5010], ['WALK', 496, 600]]}

#### Visualization

In [8]:
plot(r.json()['plan'], color = 'green')

Trip from (16.8942,52.4050) to (16.9498,52.4062) at 1678186800000. 
3 connections found. 
Best one is 32min (1038m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	438	361
2	TRAM	5010	1080
3	WALK	600	496


#### Output

In [9]:
pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
id,,,,,,,,
247,True,3,1270,254.654880,1,973,74,"[['WALK', 125, 136], ['TRAM', 332, 1469], ['WA..."
557,True,3,1869,1021.096414,1,749,279,"[['WALK', 627, 756], ['SUBWAY', 107, 1727], ['..."
948,True,3,1825,1613.782045,1,401,133,"[['WALK', 835, 1038], ['RAIL', 240, 3520], ['W..."
816,True,3,1752,525.876737,2,1204,104,"[['WALK', 353, 422], ['TRAM', 402, 1762], ['WA..."
17,True,3,1559,694.653120,0,1002,2,"[['WALK', 341, 432], ['BUS', 1002, 4407], ['WA..."


#### Full response

In [10]:
r.json()['requestParameters']

{'date': '3-7-2023',
 'mode': 'TRANSIT,WALK',
 'arriveBy': 'false',
 'fromPlace': '52.40504,16.89423',
 'toPlace': '52.40619,16.94984',
 'time': '00:0pm',
 'maxWalkDistance': '2000'}

---